In [18]:
def linear_regression(data,Target_valrable):
    import seaborn as sns
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LinearRegression
    from sklearn.feature_selection import SequentialFeatureSelector as sfs
    import numpy as np 
    from sklearn.metrics import r2_score
    from sklearn.metrics import mean_squared_error 
    
    x=list(data.columns)
    for i in x:
        if data[i].dtypes=='object':
            data.drop(i,axis=1,inplace=True)
    
   
            
    x=list(data.columns)
    ## Treating outliers
    for i in x: 
         
            q1 = data[i].quantile(0.25)  
            q3 = data[i].quantile(0.75) 
            iqr = q3-q1 
            ul = q3 + 1.5*iqr
            ll = q1 - 1.5*iqr  
            data[i].clip(upper=ul,lower=ll,inplace=True)
    
    
            
     ## filling null values
    for i in x:
        if (data[i].isnull().sum()/len(data[i])*100)>=40:
            data.drop(i,axis=1,inplace=True)
        else:
            q1 = data[i].quantile(0.25)
            q3 = data[i].quantile(0.75)
            iqr = q3 - q1
            upperL = q3+1.5*iqr
            lowerL = q1-1.5*iqr
            if data[i].min()<lowerL or data[i].max()>upperL:
                data[i].fillna(data[i].median(),inplace = True)
            else:
                data[i].fillna(data[i].mean(),inplace = True)
    
     
        
            
    
    ## Removing duplicate values
    for i in x:
        duplicate_ = data.duplicated()
    if duplicate_.any():
        data.drop_duplicates(subset=[i], inplace=True)
        
           

    ## Data dividing
    Y=pd.DataFrame(Target_valrable)
    
    X=data.drop(Y,axis=1)
    


    ## combibing the dependent and independent variable
    df=pd.concat([X, Y],axis=1)
    
    print("Exploratory Data Analysis (EDA) done successfully")
   

    ## Checking skewness
    
    s=float(Y.skew())
            
    if s!=0:
        Y=np.log(Y)
        ## Model buliding
        print("Train dataset contains 70% of the dataset and Test Dataset contains 30% of the dataset")
        X_train, X_test, y_train, y_test =train_test_split(X,Y, train_size=0.7 ,random_state=134)
        train = pd.concat([y_train, X_train], axis=1)
        lreg = LinearRegression()

        # Feature selection
        print("Total 5 features selected from the Independent Variable")
        Model = sfs(lreg, n_features_to_select = 5, direction='backward', scoring='r2',cv=5)
        Model.fit(X_train,y_train)
        f_out=list(Model.get_feature_names_out().flatten())
        print("Features selected from the independent variables are:",f_out)
        X_train = X_train.loc[:,f_out]
        train = pd.concat([y_train, X_train], axis=1)
        Model = LinearRegression()
        Model.fit(X_train,y_train)

        ## Prediction 
        train_target=train.iloc[:,0]
        train['fitted_value'] = np.round(Model.predict(X_train),2)
        train['Residual'] = np.round(train_target - train.fitted_value,2)


        r2 = r2_score(train_target,train.fitted_value)
        print('R2 score for model Performance on Train : ', np.round(r2,2)*100)



        # calculate the mean squared error
        model_mse = mean_squared_error(train_target,train['fitted_value'])
        # calulcate the root mean squared error
        model_rmse = np.sqrt(model_mse)
        print("RMSE of Train Data : ",np.exp(model_rmse))
        
        # Pediction on test data
        X_test = X_test.loc[:,f_out]
        test = pd.concat([y_test, X_test], axis=1)
        
        test_target=test.iloc[:,0]
        test['Prediction'] = np.round(Model.predict(X_test),2)
        test['Residual'] = np.round(test_target - test.Prediction,2)
        
        
        r2 = r2_score(test_target,test.Prediction)
        print('R2 score for model Performance on Test : ', np.round(r2,2)*100)
        
         
        # calculate the mean squared error
        model_mse = mean_squared_error(test_target,test['Prediction'])
        # calulcate the root mean squared error
        model_rmse = np.sqrt(model_mse)
        print("RMSE of Test Data : ",np.exp(model_rmse))
    else:
        ## Model buliding
        X_train, X_test, y_train, y_test =train_test_split(X,Y, train_size=0.7 ,random_state=134)
        train = pd.concat([y_train, X_train], axis=1)
        lreg = LinearRegression()

        # Feature selection 
        Model = sfs(lreg, n_features_to_select = 5, direction='backward', scoring='r2',cv=5)
        Model.fit(X_train,y_train)
        f_out=list(Model.get_feature_names_out().flatten())
        X_train = X_train.loc[:,f_out]
        train = pd.concat([y_train, X_train], axis=1)
        Model = LinearRegression()
        Model.fit(X_train,y_train)

        ## Prediction 
        train_target=train.iloc[:,0]
        train['fitted_value'] = np.round(Model.predict(X_train),2)
        train['Residual'] = np.round(train_target - train.fitted_value,2)


        r2 = r2_score(train_target,train.fitted_value)
        print('R2 score for model Performance on Train : ', np.round(r2,2)*100)



        # calculate the mean squared error
        model_mse = mean_squared_error(train_target,train['fitted_value'])
        # calulcate the root mean squared error
        model_rmse = np.sqrt(model_mse)

        print("RMSE of Train Data : ",np.round(model_rmse))
        
        #Prediction on test data
        # Pediction on test data
        X_test = X_test.loc[:,f_out]
        test = pd.concat([y_test, X_test], axis=1)
        
        test_target=test.iloc[:,0]
        test['Prediction'] = np.round(Model.predict(X_test),2)
        test['Residual'] = np.round(test_target - test.Prediction,2)
        
        
        r2 = r2_score(test_target,test.Prediction)
        print('R2 score for model Performance on Test : ', np.round(r2,2)*100)
        
         # calculate the mean squared error
        model_mse = mean_squared_error(test_target,test['Prediction'])
        # calulcate the root mean squared error
        model_rmse = np.sqrt(model_mse)
        print("RMSE of Test Data : ",np.round(model_rmse,2))
        